# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

# Описание данных

- children — количество детей в семье
- days_employed — общий трудовой стаж в днях
- dob_years — возраст клиента в годах
- education — уровень образования клиента
- education_id — идентификатор уровня образования
- family_status — семейное положение
- family_status_id — идентификатор семейного положения
- gender — пол клиента
- income_type — тип занятости
- debt — имел ли задолженность по возврату кредитов
- total_income — ежемесячный доход
- purpose — цель получения кредита


## Изучение данных из файла

In [2]:
import pandas as pd
from pymystem3 import Mystem

In [3]:
df = pd.read_csv('./datasets/data.csv')
display(df.head(10))
df.info()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [3]:
df.sort_values(by='dob_years').head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
6859,1,-260.992722,0,неоконченное высшее,2,гражданский брак,1,F,сотрудник,0,128246.646591,покупка недвижимости
16042,2,-176.404487,0,среднее,1,женат / замужем,0,F,сотрудник,0,137563.630693,жилье
7034,0,366067.781030,0,высшее,0,Не женат / не замужем,4,F,пенсионер,0,263121.074528,образование
21179,2,-108.967042,0,высшее,0,женат / замужем,0,M,компаньон,0,240702.007382,строительство жилой недвижимости
4147,0,-3549.117333,0,среднее,1,в разводе,3,M,сотрудник,0,108130.933212,покупка жилой недвижимости


При предварительном анализе данных, в глаза бросается столбец "days_employed" — "общий трудовой стаж в днях". В столбце используется вещественный тип данных, большинство значений имеют отрицательные значения. Если использовать данные значения, как количество дней стажа, то трудовой стаж превышает возраст клиента. Если считать, что данные указаны в формате unix time или указано количество часов трудового стажа, то расчеты тоже не корректны. При необходимости использования этих данных, необходимо будет уточнить информацию и корректность данных.

Также в столбцах "days_employed" и "total_income" имеются пропущенные значения, по 2174 записи, что составляет 10%  от всех данных. Так как эти пропуски могут исказить анализ данных, то в дальнейшем надо будет их заполнить. 

В столбце "dob_years" у 101 записях указан 0, необходимо заменить 0 на подходящие значения.

В столбце "education" данные записаны в разных регистрах.

## Предобработка данных

### Обработка пропусков

Заменим пропуски о ежемесячном доходе (столбец "total_income") методом median() для каждого типа занятости.

В столбце о возрасте клиентов (столбец "dob_years"), возьмем среднее значение возраста для каждого типа занятости.

In [4]:
employment = df['income_type'].sort_values().unique()
employment

array(['безработный', 'в декрете', 'госслужащий', 'компаньон',
       'пенсионер', 'предприниматель', 'сотрудник', 'студент'],
      dtype=object)

In [5]:
for row in employment:
    df.loc[(df['income_type']==row) & (df['total_income'].isna()), 'total_income'] = df.loc[df['income_type']==row,'total_income'].median()
    df.loc[(df['income_type']==row) & (df['dob_years']==0), 'dob_years'] = df.loc[df['income_type']==row,'dob_years'].mean()

Пропуски общего трудового стажа в днях (столбец " days_employed "), заменим средними значениями по возрасту клиента

In [6]:
year = df['dob_years'].unique() # Создадим переменную уникальными значениями возраста

In [7]:
for i in year:
    df.loc[(df['dob_years']==i) & (df['days_employed'].isna()), 'days_employed'] = df.loc[df['dob_years']==i,'days_employed'].mean()
    
df.isna().sum() 

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Были заменены все пустые значения. В столбце  "total_income", пропуски заменили методом median(). Медиану брал по сгруппированным данным типа занятости, т.к. наиболее вероятнее, что клиенты занимающиеся одинаковым типом занятости в среднем будут получать одинаковый доход.
В столбце "dob_years", 0 заменил средним возрастом клиентов сгруппированных по типу занятости . 
В столбце "days_employed ", пропуски заменили методом mean(). Среднее значение взяли по клиентам одного возраста

Данные пропуски, скорее всего появились, из-за не предоставленных  или неподтвержденных данных клиентами.

### Замена типа данных

In [8]:
df = df.astype({'dob_years':'int', 'days_employed':'int', 'total_income':'int'}, copy=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
children            21525 non-null int64
days_employed       21525 non-null int32
dob_years           21525 non-null int32
education           21525 non-null object
education_id        21525 non-null int64
family_status       21525 non-null object
family_status_id    21525 non-null int64
gender              21525 non-null object
income_type         21525 non-null object
debt                21525 non-null int64
total_income        21525 non-null int32
purpose             21525 non-null object
dtypes: int32(3), int64(4), object(5)
memory usage: 1.7+ MB


Для более удобного анализа, столбцы с вещественными типами перевели в целочисленный тип методом astype(). Методом astype() удобно производить замену на необходимый тип одновременно в нескольких столбцах. 

### Обработка дубликатов

Найдем и удалим явные дубликаты в таблице:

In [9]:
df.duplicated().sum()

54

In [10]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

Приведем значения в столбце "days_employed " к нижнему регистру методом str.lower(). Найдем и удалим имеющиеся дубликаты

In [11]:
df['education'] = df['education'].str.lower()
df.duplicated().sum()

17

In [12]:
df = df.drop_duplicates().reset_index(drop=True)
df.duplicated().sum()

0

Явные дубликаты в данных были найдены и удалены методами duplicated() и drop_duplicates(). Так же был произведен ручной поиск дубликатов с учетом регистра.

Причина появления дубликатов скорее всего является повторное обращение клиентов или дублирование информации со стороны сотрудников 

### Лемматизация

Выделим леммы в значениях столбца "purpose" с целями получения кредита

In [13]:
m = Mystem() 

df_lemmatize = pd.DataFrame(df['purpose'].unique())
df_lemmatize.set_axis(['purpose'], axis='columns', inplace=True)

def lemma_pur(lem):
    return m.lemmatize(lem['purpose'])

df_lemmatize['lemma_purpose'] = df_lemmatize.apply(lemma_pur, axis=1)

df_lemmatize.head()

,purpose,lemma_purpose
0,покупка жилья,"[покупка, , жилье, \n]"
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,дополнительное образование,"[дополнительный, , образование, \n]"
3,сыграть свадьбу,"[сыграть, , свадьба, \n]"
4,операции с жильем,"[операция, , с, , жилье, \n]"


Объединим таблицы методом merge()

In [14]:
df = df.merge(df_lemmatize, on='purpose', how='left')
df.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,lemma_purpose
0,1,-8437,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875,покупка жилья,"[покупка, , жилье, \n]"
1,1,-4024,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080,приобретение автомобиля,"[приобретение, , автомобиль, \n]"
2,0,-5623,33,среднее,1,женат / замужем,0,M,сотрудник,0,145885,покупка жилья,"[покупка, , жилье, \n]"
3,3,-4124,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628,дополнительное образование,"[дополнительный, , образование, \n]"
4,0,340266,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616,сыграть свадьбу,"[сыграть, , свадьба, \n]"


Для облегчения процесса лемматизации, создал DataFrame - df_lemmatize с уникальными значениями столбца "purpose".

### Категоризация данных

Создал словарь по данным об образовании

In [15]:
df_education = df[['education', 'education_id']]
df_education = df_education.drop_duplicates().reset_index(drop=True)
df_education

,education,education_id
0,высшее,0
1,среднее,1
2,неоконченное высшее,2
3,начальное,3
4,ученая степень,4


Создал словарь по семейному положению

In [16]:
df_family = df[['family_status', 'family_status_id']]
df_family = df_family.drop_duplicates().reset_index(drop=True)
df_family

,family_status,family_status_id
0,женат / замужем,0
1,гражданский брак,1
2,вдовец / вдова,2
3,в разводе,3
4,Не женат / не замужем,4


Категоризация по целям взятия кредита

In [17]:
mission = ['жилье', 'автомобиль', 'образование', 'свадьба', 'недвижимость']

for row in range(len(df_lemmatize)):
    for word in df_lemmatize.loc[row, 'lemma_purpose']:
        for i in mission:
            if i in word:
                df_lemmatize.loc[row, 'lemma_reduct'] = i
df_lemmatize.head()


,purpose,lemma_purpose,lemma_reduct
0,покупка жилья,"[покупка, , жилье, \n]",жилье
1,приобретение автомобиля,"[приобретение, , автомобиль, \n]",автомобиль
2,дополнительное образование,"[дополнительный, , образование, \n]",образование
3,сыграть свадьбу,"[сыграть, , свадьба, \n]",свадьба
4,операции с жильем,"[операция, , с, , жилье, \n]",жилье


In [18]:
df_lemmatize['lemma_reduct'].value_counts()

недвижимость    10
автомобиль       9
образование      9
жилье            7
свадьба          3
Name: lemma_reduct, dtype: int64

Как видно, в основном кредиты берут на операции с недвижимостью и покупку транспорта. 

## Исследовательский анализ данных

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [19]:
df['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5], dtype=int64)

В столбце "children" заменим "-1" и "20" на "1" и "2" соответственно

In [20]:
df['children'] = df['children'].replace([-1, 20], [1, 2])

In [21]:
df['children'].value_counts()

0    14091
1     4855
2     2128
3      330
4       41
5        9
Name: children, dtype: int64

In [22]:
pivot_childer = df.pivot_table(index=['children'], columns='debt', values='family_status', aggfunc='count')
pivot_childer['pct'] = pivot_childer[1] / (pivot_childer[1] + pivot_childer[0])  * 100
pivot_childer

debt,0,1,pct
children,,,
0,13028.0,1063.0,7.543822
1,4410.0,445.0,9.165808
2,1926.0,202.0,9.492481
3,303.0,27.0,8.181818
4,37.0,4.0,9.756098
5,9.0,NaN,NaN


Можно сделать вывод, что клиенты без детей чаще берут кредиты, но задолженность имеется только у 7,5% клиентов. А у клиентов с детьми задолженность по возврату кредитов в 9,5% случаев. Клиенты с детьми, чаще имеют задолженность 

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
pivot_family = df.pivot_table(index=['family_status'], columns='debt', values='family_status_id', aggfunc='count')
pivot_family['pct'] = pivot_family[1] / (pivot_family[1] + pivot_family[0])  * 100
pivot_family

debt,0,1,pct
family_status,,,
Не женат / не замужем,2536,274,9.750890
в разводе,1110,85,7.112971
вдовец / вдова,896,63,6.569343
гражданский брак,3763,388,9.347145
женат / замужем,11408,931,7.545182


Чаще всего берут кредиты женатые клиенты, при задолженности у 7,5% клиентов. Задолженность чаще у не женатые и в гражданском браке клиентов, в 9,7 и 9,3 процентах. А вдовцы реже всего имеют задолженность, всего у 6,6%. 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
income_cat = [[50000, 100000], [100000, 150000], [150000, 200000], [200000, 250000]]

for i in range(len(df)):
    for row in income_cat:
        if row[0] < df.loc[i, 'total_income'] <= row[1]:
            df.loc[i, 'income'] = 'До ' + str(row[1])
        elif df.loc[i, 'total_income'] <= 50000:
            df.loc[i, 'income'] = 'До 50000'
        elif df.loc[i, 'total_income'] > 250000:
            df.loc[i, 'income'] = 'Более 250000'

In [25]:
pivot_income = df.pivot_table(index=['income'], columns='debt', values='family_status_id', aggfunc='count')
pivot_income['pct'] = pivot_income[1] / (pivot_income[1] + pivot_income[0])  * 100
pivot_income

debt,0,1,pct
income,,,
Более 250000,2619,194,6.896552
До 100000,3760,331,8.090931
До 150000,6536,624,8.715084
До 200000,4359,405,8.501259
До 250000,2090,164,7.275954
До 50000,349,23,6.182796


Больше всего кредиты брали клиенты с доходом от 100000 до 150000, и они имеют наибольший процент задолженности 8.7%. Из данных видно, что менее всего задолженностей по кредитам (6,2%), у клиентов с наименьшим доходом в месяц до 50000.

Можно выделить, что клиенты с наименьшими и наибольшими доходами за месяц, возвращают кредиты в срок. А клиенты с доходами 50000 – 200000,  имеют больше задолженностей за кредиты 

- Как разные цели кредита влияют на его возврат в срок?

In [26]:
table_pursore = df.merge(df_lemmatize, on='purpose', how='left')

In [27]:
pivot_purpose = table_pursore.pivot_table(index=['lemma_reduct'], columns='debt', values='family_status_id', aggfunc='count')
pivot_purpose['pct'] = pivot_purpose[1] / (pivot_purpose[1] + pivot_purpose[0])  * 100
pivot_purpose

debt,0,1,pct
lemma_reduct,,,
автомобиль,3903,403,9.359034
жилье,4152,308,6.905830
недвижимость,5877,474,7.463392
образование,3643,370,9.220035
свадьба,2138,186,8.003442


Чаще кредиты брали на недвижимость (жилую и коммерческую) и по этим кредитам меньше всего задолженностей (6,9% и 7,5% соответственно). Чаще не возвращают кредиты за транспорт и образование, 9,3% и 9,2%. 

## Вывод

Проверили зависимости возврата кредита в срок и данными клиентов:

1.	Зависимость между наличием детей и возвратом кредита в срок.

Можно сказать, что клиенты без детей чаще возвращают кредиты в срок.

2.	Зависимость между семейным положением и возвратом кредита в срок:

Клиенты, находящиеся в официальном браке, в разводе и вдовцы лучше возвращают кредиты. Нежели не женатые или находящиеся в гражданском браке 

3.	Зависимость между уровнем дохода и возвратом кредита в срок.

Клиенты с наименьшими и наибольшими доходами за месяц, чаще возвращают кредиты в срок. А клиенты с доходами 50000 – 200000 имеют больше всего задолженностей за кредиты 

4.	Как разные цели кредита влияют на его возврат в срок.

Клиенты приобретающие любую недвижимость в кредит, лучше возвращают кредиты в срок. Чаще имеют задолженность за кредиты на транспорт и образование
